In [1]:
import pandas as pd

In [55]:
%%capture
pip install qgrid

In [30]:
#fantasy data for all regular season weeks
weekly_data = pd.read_csv("weekly_test.csv")

In [31]:
#df for every year week position with def team
grouped_data = (
    weekly_data.groupby(['year', 'week', 'posteam', 'defteam', 'position'], as_index=False)
    .agg({'half_ppr': 'sum'})
)

In [32]:
# Step 1: Aggregate total sum and count by year, posteam, and position
total_agg = (
    grouped_data.groupby(['year', 'posteam', 'position'], as_index=False)
    .agg(total_half_ppr=('half_ppr', 'sum'), count=('half_ppr', 'count'))
)

In [33]:
# Step 2: Merge total_agg back into the original grouped_data
grouped_data = grouped_data.merge(
    total_agg, 
    on=['year', 'posteam', 'position'], 
    how='left'
)

In [34]:
# Step 3: Calculate leave-one-out average
grouped_data['team_position_avg'] = (
    (grouped_data['total_half_ppr'] - grouped_data['half_ppr']) /
    (grouped_data['count'] - 1)
)
# Step 4: Drop intermediate columns if needed
grouped_data.drop(columns=['total_half_ppr', 'count'], inplace=True)

In [35]:
grouped_data['adjustment'] = grouped_data['half_ppr'] - grouped_data['team_position_avg']

In [36]:
adjustment_avg = (
    grouped_data.groupby(['year', 'defteam', 'position'], as_index=False)
    .agg(avg_adjustment=('adjustment', 'mean'))
)


In [37]:
adjustment_avg

,year,defteam,position,avg_adjustment
0,2003,ARI,QB,4.563084
1,2003,ARI,RB,-0.049500
2,2003,ARI,TE,-0.591889
3,2003,ARI,WR,2.831583
4,2003,ATL,QB,2.453000
...,...,...,...,...
2811,2024,TEN,WR,-5.639933
2812,2024,WAS,QB,-1.123111
2813,2024,WAS,RB,1.959597
2814,2024,WAS,TE,-0.135861


In [38]:
adjustment_avg_2024 = adjustment_avg[adjustment_avg['year'] == 2024]

In [39]:
# Pivot the table to show `defteam` as rows, `position` as columns, and `avg_adjustment` as values
pivot_table_2024 = adjustment_avg_2024.pivot(index='defteam', columns='position', values='avg_adjustment')
pivot_table_2024 = pivot_table_2024.sort_index()

In [40]:
def color_map(val):
    if pd.notnull(val):  # Check for non-NaN values
        # Normalize the value between 0 and 1
        norm_val = (val - pivot_table_2024.min().min()) / (pivot_table_2024.max().max() - pivot_table_2024.min().min())
        # Map normalized value to blue-green gradient
        blue = int(255 * (1 - norm_val))  # Higher values reduce blue
        green = int(255 * norm_val)  # Higher values increase green
        return f'background-color: rgb(0, {green}, {blue})'  # Use only green and blue channels
    return ''  # Leave NaN values uncolored

In [41]:
styled_table = pivot_table_2024.style.map(color_map)

In [42]:
styled_table

position,QB,RB,TE,WR
defteam,,,,
ARI,-0.973844,-0.540000,-1.234444,-1.092778
ATL,2.397474,-2.080707,0.030788,5.451515
BAL,3.228627,-1.072222,0.398384,9.263758
BUF,-2.019959,2.923131,-0.741313,-1.546384
CAR,3.029711,9.704000,4.117450,-1.267356
CHI,-3.928978,1.947778,0.147444,-5.048578
CIN,3.997414,-1.845051,2.296187,3.346364
CLE,-0.575978,-2.815223,0.677911,3.898111
DAL,2.356111,4.240334,-1.706872,0.099689


In [63]:
# Export the styled table as an HTML file
html_table = styled_table.to_html()  # Use to_html instead of render()
with open("styled_table.html", "w") as file:
    file.write(html_table)